In [1]:
# !pip install torchvision

In [2]:
# !pip uninstall typing_extensions
# !pip install typing_extensions==4.11.0

In [3]:
!pip install typing_extensions>=4.3 --upgrade

In [4]:
# !pip install --upgrade pydantic

In [5]:
!pip install typing_extensions==4.12.2 --upgrade
# pip install typing_extensions==4.7.1 --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.13.0
    Uninstalling typing_extensions-4.13.0:
      Successfully uninstalled typing_extensions-4.13.0


In [6]:
from typing_extensions import TypeIs

**Preprocessing - flipping, resizing, rotation, gamma correction**

In [7]:
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = [os.path.join(image_dir, x) for x in os.listdir(image_dir) if x.endswith('.png')]
        self.masks = [os.path.join(mask_dir, x) for x in os.listdir(mask_dir) if 'Annotation' in x]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        mask_path = self.masks[idx]
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        return image, mask

# Define transformations including geometric and intensity-based augmentations
transform = transforms.Compose([
    transforms.Resize((572, 572)),  # Resize images to match U-Net expected input
    transforms.RandomHorizontalFlip(),  # Random horizontal flipping
    transforms.RandomVerticalFlip(),  # Random vertical flipping
    transforms.RandomRotation(20),  # Random rotations between -20 to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Random brightness and contrast adjustments
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.pow(0.5))  # Gamma correction with gamma=0.5
])

# Initialize dataset
full_dataset = CustomDataset('denoised_training_set', 'masked_annotations', transform=transform)

# Splitting the dataset into train and validation sets
train_size = int(0.8 * len(full_dataset))
validation_size = len(full_dataset) - train_size
train_dataset, validation_dataset = random_split(full_dataset, [train_size, validation_size])

# Create separate dataloaders for train and validation datasets
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=10, shuffle=False)

In [8]:
# Now, train_loader and validation_loader can be used in training and validation phases.

**Unet with resnet101 as backbone**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class ResConv(nn.Module):
    """ Convolution block for U-Net with repeated convolutions and ReLU activations. """
    def __init__(self, in_ch, out_ch):
        super(ResConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UpConv(nn.Module):
    """ Upsampling block for U-Net, using bilinear interpolation and convolution. """
    def __init__(self, in_ch, out_ch):
        super(UpConv, self).__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = ResConv(in_ch, out_ch)

    def forward(self, from_down, from_up):
        from_up = self.up(from_up)
        diffY = from_down.size()[2] - from_up.size()[2]
        diffX = from_down.size()[3] - from_up.size()[3]
        from_up = F.pad(from_up, [diffX // 2, diffX - diffX // 2,
                                  diffY // 2, diffY - diffY // 2])
        x = torch.cat([from_down, from_up], dim=1)
        return self.conv(x)

class UNetResNet101(nn.Module):
    def __init__(self, n_classes=1):
        super(UNetResNet101, self).__init__()
        base_model = models.resnet101(pretrained=True)
        self.base_layers = list(base_model.children())
        
        # Extract layers from ResNet101
        self.layer0 = nn.Sequential(*self.base_layers[:3])  # conv1, bn1, relu
        self.maxpool = self.base_layers[3]
        self.layer1 = self.base_layers[4]  # Output: 256 channels
        self.layer2 = self.base_layers[5]  # Output: 512 channels
        self.layer3 = self.base_layers[6]  # Output: 1024 channels
        self.layer4 = self.base_layers[7]  # Output: 2048 channels

        # Decoder (make sure the channel numbers match the skip connection outputs)
        self.up4 = UpConv(2048 + 1024, 1024)  # Concatenating x3 (1024) and x4 (2048) -> 3072 channels
        self.up3 = UpConv(1024 + 512, 512)    # Concatenating x2 (512) and previous output (1024) -> 1536 channels
        self.up2 = UpConv(512 + 256, 256)     # Concatenating x1 (256) and previous output (512) -> 768 channels
        self.up1 = UpConv(256 + 64, 64)       # Concatenating x0 (64) and previous output (256) -> 320 channels

        # Final upsampling and output convolution to match input size
        self.final_up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.final_conv = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        # Encoder: get intermediate features for skip connections
        x0 = self.layer0(x)       # Early features, e.g., 64 channels
        x1 = self.maxpool(x0)
        x1 = self.layer1(x1)      # 256 channels
        x2 = self.layer2(x1)      # 512 channels
        x3 = self.layer3(x2)      # 1024 channels
        x4 = self.layer4(x3)      # 2048 channels

        # Decoder: use skip connections from intermediate features
        x = self.up4(x3, x4)      # Upsample: x3 (from_down) + x4 (from_up)
        x = self.up3(x2, x)       # Upsample: x2 + output of previous block
        x = self.up2(x1, x)       # Upsample: x1 + output of previous block
        x = self.up1(x0, x)       # Upsample: x0 + output of previous block

        x = self.final_up(x)      # Final upsampling to the original size
        x = self.final_conv(x)
        return torch.sigmoid(x)

**Dice loss + Binary cross-entropy loss**

In [4]:
import torch.nn.functional as F

class DiceBCELoss(nn.Module):
    def __init__(self):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        # Flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

**Training Loop**

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [16]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
from sklearn.metrics import f1_score

# Assuming UNetResNet101 and DiceBCELoss are already imported
model = UNetResNet101().to(device)  # Ensure your model is the one with ResNet-101
loss_function = DiceBCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

num_epochs = 50  # Set the number of epochs you want to train for

for epoch in range(num_epochs):
    # Training loop
    model.train()
    running_loss = 0.0
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Compute training metrics in evaluation mode
    model.eval()
    train_loss = 0.0
    all_train_preds = []
    all_train_targets = []
    with torch.no_grad():
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = loss_function(outputs, masks)
            train_loss += loss.item()
            # Threshold outputs and targets at 0.5 to obtain binary predictions
            preds = (outputs > 0.5).float()
            binary_masks = (masks > 0.5).float()
            all_train_preds.append(preds.cpu().numpy().flatten())
            all_train_targets.append(binary_masks.cpu().numpy().flatten())
    all_train_preds = np.concatenate(all_train_preds)
    all_train_targets = np.concatenate(all_train_targets)
    train_f1 = f1_score(all_train_targets, all_train_preds)

    # Compute validation metrics
    val_loss = 0.0
    all_val_preds = []
    all_val_targets = []
    with torch.no_grad():
        for images, masks in validation_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = loss_function(outputs, masks)
            val_loss += loss.item()
            preds = (outputs > 0.5).float()
            binary_masks = (masks > 0.5).float()
            all_val_preds.append(preds.cpu().numpy().flatten())
            all_val_targets.append(binary_masks.cpu().numpy().flatten())
    all_val_preds = np.concatenate(all_val_preds)
    all_val_targets = np.concatenate(all_val_targets)
    val_f1 = f1_score(all_val_targets, all_val_preds)

    # Print epoch summary with both training and validation metrics
    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {running_loss/len(train_loader):.4f}, Train F1: {train_f1:.4f}, '
          f'Validation Loss: {val_loss/len(validation_loader):.4f}, Validation F1: {val_f1:.4f}')
    
    # Adjust learning rate based on the validation loss
    scheduler.step(val_loss/len(validation_loader))

# Save the trained model
torch.save(model.state_dict(), 'unet_resnet101_model.pth')

Epoch 1/50, Train Loss: 0.8501, Train F1: 0.7885, Validation Loss: 0.8071, Validation F1: 0.7872
Epoch 2/50, Train Loss: 0.7883, Train F1: 0.7998, Validation Loss: 0.7780, Validation F1: 0.7966
Epoch 3/50, Train Loss: 0.7679, Train F1: 0.7968, Validation Loss: 0.7792, Validation F1: 0.7940
Epoch 4/50, Train Loss: 0.7699, Train F1: 0.8015, Validation Loss: 0.7594, Validation F1: 0.8000
Epoch 5/50, Train Loss: 0.7587, Train F1: 0.8008, Validation Loss: 0.7665, Validation F1: 0.7956
Epoch 6/50, Train Loss: 0.7553, Train F1: 0.8023, Validation Loss: 0.7756, Validation F1: 0.7999
Epoch 7/50, Train Loss: 0.7583, Train F1: 0.8029, Validation Loss: 0.7651, Validation F1: 0.7999
Epoch 8/50, Train Loss: 0.7492, Train F1: 0.8036, Validation Loss: 0.7808, Validation F1: 0.8008
Epoch 9/50, Train Loss: 0.7628, Train F1: 0.8045, Validation Loss: 0.7625, Validation F1: 0.8023
Epoch 10/50, Train Loss: 0.7531, Train F1: 0.8028, Validation Loss: 0.7544, Validation F1: 0.7996
Epoch 11/50, Train Loss: 0.76

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = UNetResNet101(n_classes=1).to(device)
# dummy_input = torch.rand(1, 3, 224, 224).to(device)  # Adjust input size as necessary
# output = model(dummy_input)
# print("Output size:", output.size())

**Running Model on test data**

In [6]:
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# Device configuration
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define a test dataset class
class TestDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = sorted([os.path.join(image_dir, x) for x in os.listdir(image_dir) if x.endswith('.png')])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, image_path

# Define deterministic transforms for test data
test_transform = transforms.Compose([
    transforms.Resize((572, 572)),       # Resize to match model input size
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.pow(0.5))  # Gamma correction (same as training)
])

# Set the directory where your test images are stored
test_dir = 'denoised_test_set'  # Adjust this to your actual test directory path

# Create the test dataset and dataloader
test_dataset = TestDataset(test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Instantiate your model (assumed defined in the notebook)
model = UNetResNet101(n_classes=1).to(device)

# Load the saved model weights
model.load_state_dict(torch.load('unet_resnet101_model.pth', map_location=device))
model.eval()

# Create an output directory for the segmentation results
output_dir = 'output_segmentations'
os.makedirs(output_dir, exist_ok=True)

# Inference loop: run the model on each test image and save the segmentation mask
for image, image_path in test_loader:
    image = image.to(device)
    with torch.no_grad():
        output = model(image)  # Get the probability map from the model (sigmoid already applied)
        # Threshold the probability map to obtain a binary segmentation mask
        seg_mask = (output > 0.5).float()
    
    # Convert tensor to NumPy array and scale to 0-255 for visualization/saving
    seg_mask_np = seg_mask.cpu().numpy().squeeze() * 255

    # Generate output filename based on input image name
    base_name = os.path.basename(image_path[0])
    output_filename = os.path.join(output_dir, f"seg_{base_name}")
    
    # Save the segmentation mask image using OpenCV
    cv2.imwrite(output_filename, seg_mask_np.astype('uint8'))
    
    # Optionally, display the segmentation mask using matplotlib
#     plt.imshow(seg_mask_np, cmap='gray')
#     plt.title(f"Segmentation: {base_name}")
#     plt.axis('off')
#     plt.show()
    
    print(f"Saved segmentation for {base_name} at {output_filename}")

/usr/lib/python3/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/lib/python3/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Saved segmentation for 000_HC.png at output_segmentations/seg_000_HC.png
Saved segmentation for 001_HC.png at output_segmentations/seg_001_HC.png
Saved segmentation for 002_HC.png at output_segmentations/seg_002_HC.png
Saved segmentation for 003_HC.png at output_segmentations/seg_003_HC.png
Saved segmentation for 004_HC.png at output_segmentations/seg_004_HC.png
Saved segmentation for 005_HC.png at output_segmentations/seg_005_HC.png
Saved segmentation for 006_HC.png at output_segmentations/seg_006_HC.png
Saved segmentation for 007_HC.png at output_segmentations/seg_007_HC.png
Saved segmentation for 008_HC.png at output_segmentations/seg_008_HC.png
Saved segmentation for 009_HC.png at output_segmentations/seg_009_HC.png
Saved segmentation for 010_HC.png at output_segmentations/seg_010_HC.png
Saved segmentation for 011_HC.png at output_segmentations/seg_011_HC.png
Saved segmentation for 012_HC.png at output_segmentations/seg_012_HC.png
Saved segmentation for 013_HC.png at output_segment

Saved segmentation for 113_HC.png at output_segmentations/seg_113_HC.png
Saved segmentation for 114_HC.png at output_segmentations/seg_114_HC.png
Saved segmentation for 115_HC.png at output_segmentations/seg_115_HC.png
Saved segmentation for 116_HC.png at output_segmentations/seg_116_HC.png
Saved segmentation for 117_HC.png at output_segmentations/seg_117_HC.png
Saved segmentation for 118_HC.png at output_segmentations/seg_118_HC.png
Saved segmentation for 119_HC.png at output_segmentations/seg_119_HC.png
Saved segmentation for 120_HC.png at output_segmentations/seg_120_HC.png
Saved segmentation for 121_HC.png at output_segmentations/seg_121_HC.png
Saved segmentation for 122_HC.png at output_segmentations/seg_122_HC.png
Saved segmentation for 123_HC.png at output_segmentations/seg_123_HC.png
Saved segmentation for 124_HC.png at output_segmentations/seg_124_HC.png
Saved segmentation for 125_HC.png at output_segmentations/seg_125_HC.png
Saved segmentation for 126_HC.png at output_segment

Saved segmentation for 228_HC.png at output_segmentations/seg_228_HC.png
Saved segmentation for 229_HC.png at output_segmentations/seg_229_HC.png
Saved segmentation for 230_HC.png at output_segmentations/seg_230_HC.png
Saved segmentation for 231_HC.png at output_segmentations/seg_231_HC.png
Saved segmentation for 232_HC.png at output_segmentations/seg_232_HC.png
Saved segmentation for 233_HC.png at output_segmentations/seg_233_HC.png
Saved segmentation for 234_HC.png at output_segmentations/seg_234_HC.png
Saved segmentation for 235_HC.png at output_segmentations/seg_235_HC.png
Saved segmentation for 236_HC.png at output_segmentations/seg_236_HC.png
Saved segmentation for 237_HC.png at output_segmentations/seg_237_HC.png
Saved segmentation for 238_HC.png at output_segmentations/seg_238_HC.png
Saved segmentation for 239_HC.png at output_segmentations/seg_239_HC.png
Saved segmentation for 240_HC.png at output_segmentations/seg_240_HC.png
Saved segmentation for 241_HC.png at output_segment

**Morphological Opening and Closing + Canny edge Detector**

In [7]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Define input and output folders
input_folder = 'output_segmentations'
output_folder = 'output_edges'
os.makedirs(output_folder, exist_ok=True)

# Define a structuring element for the morphological operations
kernel = np.ones((5, 5), np.uint8)

# Loop over all segmented images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        img_path = os.path.join(input_folder, filename)
        # Read the segmentation image in grayscale
        seg_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # Apply morphological opening (erosion followed by dilation) to remove small artifacts
        opened = cv2.morphologyEx(seg_img, cv2.MORPH_OPEN, kernel)
        # Then apply morphological closing (dilation followed by erosion) to fill small holes
        closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)
        
        # Apply Canny edge detector to extract the contour
        # Adjust thresholds as necessary (here, 50 and 150 are example values)
        edges = cv2.Canny(closed, 50, 150)
        
        # Save the edge-detected image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, edges)
        
        # Optionally, display the original segmentation, post-morphology, and edge image side by side
#         plt.figure(figsize=(12, 4))
#         plt.subplot(1, 3, 1)
#         plt.imshow(seg_img, cmap='gray')
#         plt.title('Original Segmentation')
#         plt.axis('off')
        
#         plt.subplot(1, 3, 2)
#         plt.imshow(closed, cmap='gray')
#         plt.title('After Morphological Ops')
#         plt.axis('off')
        
#         plt.subplot(1, 3, 3)
#         plt.imshow(edges, cmap='gray')
#         plt.title('Canny Edges')
#         plt.axis('off')
        
#         plt.show()
        print(f"Processed and saved edge image for: {filename}")

Processed and saved edge image for: seg_306_HC.png
Processed and saved edge image for: seg_301_HC.png
Processed and saved edge image for: seg_189_HC.png
Processed and saved edge image for: seg_274_HC.png
Processed and saved edge image for: seg_033_HC.png
Processed and saved edge image for: seg_195_HC.png
Processed and saved edge image for: seg_147_HC.png
Processed and saved edge image for: seg_224_HC.png
Processed and saved edge image for: seg_296_HC.png
Processed and saved edge image for: seg_204_HC.png
Processed and saved edge image for: seg_318_HC.png
Processed and saved edge image for: seg_270_HC.png
Processed and saved edge image for: seg_010_HC.png
Processed and saved edge image for: seg_262_HC.png
Processed and saved edge image for: seg_117_HC.png
Processed and saved edge image for: seg_008_HC.png
Processed and saved edge image for: seg_278_HC.png
Processed and saved edge image for: seg_094_HC.png
Processed and saved edge image for: seg_186_HC.png
Processed and saved edge image 

Processed and saved edge image for: seg_252_HC.png
Processed and saved edge image for: seg_124_HC.png
Processed and saved edge image for: seg_131_HC.png
Processed and saved edge image for: seg_055_HC.png
Processed and saved edge image for: seg_300_HC.png
Processed and saved edge image for: seg_328_HC.png
Processed and saved edge image for: seg_223_HC.png
Processed and saved edge image for: seg_295_HC.png
Processed and saved edge image for: seg_259_HC.png
Processed and saved edge image for: seg_265_HC.png
Processed and saved edge image for: seg_092_HC.png
Processed and saved edge image for: seg_208_HC.png
Processed and saved edge image for: seg_181_HC.png
Processed and saved edge image for: seg_179_HC.png
Processed and saved edge image for: seg_331_HC.png
Processed and saved edge image for: seg_273_HC.png
Processed and saved edge image for: seg_080_HC.png
Processed and saved edge image for: seg_193_HC.png
Processed and saved edge image for: seg_018_HC.png
Processed and saved edge image 

**Ellipse fiting**

In [9]:
import csv
import cv2
import numpy as np
import os
import math

# --- Step 1: Load the pixel size information from the CSV file ---
# Assume your CSV file 'test_set_pixel_size.csv' has at least these columns:
# filename,pixel_size_mm
pixel_size_dict = {}
with open('test_set_pixel_size.csv', mode='r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        # Adjust the column names if they are different in your CSV file.
        filename_csv = row['filename']
        pixel_size_mm = float(row['pixel size(mm)'])
        pixel_size_dict[filename_csv] = pixel_size_mm

# --- Step 2: Process the edge images and fit ellipses ---
edges_folder = 'output_edges'
csv_output = 'ellipse_results.csv'
header = ["filename", "center_x_mm", "center_y_mm", "semi_axes_a_mm", "semi_axes_b_mm", "angle_rad"]
rows = []

for filename in sorted(os.listdir(edges_folder)):
    if filename.endswith('.png'):
        filepath = os.path.join(edges_folder, filename)
        # Read the edge image in grayscale
        edge_img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        
        # Find contours in the edge image
        contours, _ = cv2.findContours(edge_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) == 0:
            print(f"No contours found in {filename}. Skipping.")
            continue
        
        # Choose the largest contour (assumed to be the head contour)
        largest_contour = max(contours, key=cv2.contourArea)
        
        # cv2.fitEllipse requires at least 5 points
        if len(largest_contour) < 5:
            print(f"Not enough points for ellipse fitting in {filename}. Skipping.")
            continue
        
        # Fit ellipse to the largest contour
        ellipse = cv2.fitEllipse(largest_contour)
        # ellipse returns ((center_x, center_y), (full_axis_length_a, full_axis_length_b), angle_in_degrees)
        center, axes, angle = ellipse
        # Compute semi-axes (the axes given are the full lengths)
        semi_a = axes[0] / 2.0  # semi-major axis in pixels
        semi_b = axes[1] / 2.0  # semi-minor axis in pixels

        # --- Step 3: Look up the pixel conversion factor for this image ---
        # The filenames in the CSV are expected to be like "001_HC.png"
        # and our edge images are named "seg_001_HC.png". Remove the "seg_" prefix.
        base_filename = filename.replace("seg_", "", 1)
        if base_filename in pixel_size_dict:
            pixel_to_mm = pixel_size_dict[base_filename]
        else:
            print(f"Pixel size for {base_filename} not found in CSV. Skipping.")
            continue

        # Convert measurements from pixels to millimeters
        center_x_mm = center[0] * pixel_to_mm
        center_y_mm = center[1] * pixel_to_mm
        semi_a_mm = semi_a * pixel_to_mm
        semi_b_mm = semi_b * pixel_to_mm
        
        # Convert angle from degrees to radians
        angle_rad = math.radians(angle)
        
        # Append the result: filename, center_x_mm, center_y_mm, semi_axes_a_mm, semi_axes_b_mm, angle_rad
        rows.append([base_filename, center_x_mm, center_y_mm, semi_a_mm, semi_b_mm, angle_rad])

# --- Step 4: Write results to a CSV file ---
with open(csv_output, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

print(f"CSV file '{csv_output}' saved with {len(rows)} rows.")

CSV file 'ellipse_results.csv' saved with 335 rows.
